# Medlytics Final Project Notebook

## Importing Libraries

In [1]:
# Import basic system and numerical packages
import os
import re
!pip install numpy==1.22
import numpy as np
import pandas as pd

# Import statistical and signal processing functions
from scipy import signal
import scipy.stats.mstats as mstats
from sklearn import metrics

# Import the scikit-learn functions and models
from sklearn.model_selection import train_test_split
from sklearn import tree

# Import audio i/o and playing functions
import scipy.io.wavfile
from IPython.display import Audio

# Import plotting functionality
import matplotlib
import matplotlib.pyplot as plt
import graphviz
import seaborn as sns
sns.set()

#librosa
!sudo apt-get update && sudo apt-get install -y libsndfile1
!pip install librosa
import librosa

from pydub import AudioSegment
from scipy.io import wavfile

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://security.debian.org/debian-security buster/updates InRelease




libsndfile1 is already the newest version (1.0.28-6+deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
/root/venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Data Processing

In [2]:
cols = ["positive", "neutral", "negative", "label"]
data = pd.DataFrame()
val_data = pd.DataFrame()

In [3]:
def features(data):
    arr = dict()
    strs = ['neg','neu','pos']
    for i in range(3):
        arr['bandwidth_'+strs[i]] = librosa.feature.spectral_bandwidth(y=data[i]).mean()
        arr['flatness_'+strs[i]] = librosa.feature.spectral_flatness(y=data[i]).mean()
        arr['centroid_'+strs[i]] = librosa.feature.spectral_centroid(y=data[i]).mean()
        arr['rolloff_'+strs[i]] = librosa.feature.spectral_rolloff(y=data[i]).mean()
        arr['contrast_'+strs[i]] = librosa.feature.spectral_contrast(y=data[i]).mean()
        arr['mfcc_'+strs[i]] = librosa.feature.mfcc(y=data[i]).mean()
        arr['zcr_'+strs[i]] = librosa.feature.zero_crossing_rate(y=data[i]).mean()
    return arr

test = AudioSegment.from_wav('MedlyticsFinalProject/Training/t_1/negative_out.wav')
test = sound.set_channels(1)
test.export('MedlyticsFinalProject/Training/t_1/negative_out.wav', format="wav")
y, _ = librosa.load('MedlyticsFinalProject/Training/t_1/negative_out.wav', sr = 16000)
features(y)

In [4]:
count = 0

#!git clone https://github.com/DavidWangBWSI/MedlyticsFinalProject.git

filenames = ['/negative_out.wav', '/neutral_out.wav', '/positive_out.wav']

for i in range(1,112):

    try:
        path = 'Training/t_'+str(i)
        files = []
        for name in filenames:
            # Convert wav from stereo to mono
            sound = AudioSegment.from_wav(path+name)
            sound = sound.set_channels(1)
            sound.export(path+name, format="wav")
 
            y, _ = librosa.load(path+name, sr = 16000)
            files.append(y)

        row = features(files)

        with open(path+'/new_label.txt') as f:
            label = float(f.readline())
            row['label'] = int(label>=53)
        count+=1
        data = data.append(row, ignore_index=True)
    except:
        continue

/root/venv/lib/python3.9/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


In [5]:
#!git clone https://github.com/DavidWangBWSI/MedlyticsFinalProject.git

filenames = ['/negative_out.wav', '/neutral_out.wav', '/positive_out.wav']

for i in range(1,112):

    try:
        path = 'Validation/v_'+str(i)
        files = []
        for name in filenames:
            # Convert wav from stereo to mono
            sound = AudioSegment.from_wav(path+name)
            sound = sound.set_channels(1)
            sound.export(path+name, format="wav")
 
            y, _ = librosa.load(path+name, sr = 16000)
            files.append(y)

        row = features(files)

        with open(path+'/new_label.txt') as f:
            label = float(f.readline())
            row['label'] = int(label>=53)
        count+=1
        data = data.append(row, ignore_index=True)

    except:
        continue

/root/venv/lib/python3.9/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
/root/venv/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=0
  return f(*args, **kwargs)
/root/venv/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [7]:
columns = ['bandwidth_neg','bandwidth_neu','bandwidth_pos','flatness_neg','flatness_neu','flatness_pos','centroid_neg','centroid_neu','centroid_pos', 'rolloff_neg', 'contrast_neg', 'mfcc_neg', 'zcr_neg', 'rolloff_neu', 'contrast_neu', 'mfcc_neu', 'zcr_neu', 'rolloff_pos', 'contrast_pos', 'mfcc_pos', 'zcr_pos']
X = data[columns]

y_labels = data['label']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.15, random_state=0)

In [10]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [24]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(class_weight = 'auto')
logreg.fit(X_train,y_train)

y_pred = logreg.predict(X_test)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [26]:
from sklearn.svm import SVC 

# Creating a model with sklearn's SVC
svm = SVC(gamma=.1, C=1)

# Training/fitting a model with training data
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

In [21]:
! pip install xgboost
from xgboost import XGBClassifier

learning_rate=0.0005
n_estimators=1000
max_depth=7
subsample=1
colsample_bytree=1
gamma=1

xgb = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, subsample=subsample, colsample_bytree=colsample_bytree, gamma=gamma)
xgb.fit(X_train,y_train)

y_pred = xgb.predict(X_test)

In [30]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Confusion Matrix: \n", confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
print("Validation Accuracy: ", accuracy_score(y_test, y_pred))

Confusion Matrix: 
 [[18  2]
 [ 4  0]]
Validation Accuracy:  0.75


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=311e6adc-3eb1-4202-832e-d715f09028ff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>